In [1]:
import numpy as np
from StrengthClasses.glulam_strength_classes import glulam

In [2]:
glulam_class = 'GL28h'
kmod = 0.9
gammaM = 1.45

# Sollecitazione di progetto:
Q = 3.0815 #kN/m == N/mm
SLE_rara_perm = 0.23+0.545 # g1 e g2
SLE_rara_acc1 = 1.31 #carico accidentale principale
# SLE_rara_acc2 =  eventualmente aggiungerlo e de-commentare nella W_net

# Per gli SLE
k_def = 0.6
psi2_1 = 0.
# psi0_2 =
# psi2_2 =

b = 160
h = 200
l = 5000
l_appoggio = 234 # Per la verifica compressione 90 appoggio

In [3]:
# SLU
M_d = Q * l**2 / 8
V_d = Q *l / 2

print('M_d:', M_d/10**6, 'kNm')
print('V_d:', V_d/10**3, 'kN')

M_d: 9.6296875 kNm
V_d: 7.70375 kN


In [4]:
# Prende le proprietà del materiale dal dizionario che c'è nell'altro file
f_mk, f_t0k, f_t90k, f_c0k, f_c90k, f_vk, f_rk, \
E0mean, E005, E90mean, E9005, Gmean, G05, Grmean, Gr05, \
rho_k, rho_mean = glulam(glulam_class)

f_md, f_t0d, f_t90d, f_c0d, f_c90d, f_vd, f_rd = \
(kmod * X_k / gammaM for X_k in glulam(glulam_class)[0:7])

print(f_md, f_t0d, f_t90d, f_c0d, f_c90d, f_vd, f_rd)

17.379310344827587 13.841379310344829 0.3103448275862069 17.379310344827587 1.5517241379310345 2.1724137931034484 0.7448275862068966


### Flessione
$$ \sigma_{m,d} = \frac{M_d}{W}\leq f_{m,d} = \frac{k_{mod} \cdot f_{m,k}}{\gamma_M} $$ 

sbandamento
$$ \sigma_{m,d} \leq k_{crit} \cdot f_{m,d} $$ 

In [5]:
W = b * h**2 / 6 #sezioni rettangolari
print(W)
J = b*h**3 / 12
print(J)

sigma_md = M_d / W
print(sigma_md)

1066666.6666666667
106666666.66666667
9.02783203125


## Stabilità flessotorsionale -- formula semplificata EC

In [18]:
l_eff = 0.9*l + 2*h 

sigma_mcrit = (0.78 * b**2 * E005) / (h * l_eff) 
lambda_relm = np.sqrt(f_mk/sigma_mcrit)
if lambda_relm < 0.75:
    k_crit = 1
elif lambda_relm < 1.14:
    k_crit = 1.56 - .75*lambda_relm
else:
    k_crit = 1/(lambda_relm**2)

print('h: ', h)
print('l_eff: ', l_eff)
print('l_eff/l: ', l_eff/l)
print('sigma_mcrit: ', sigma_mcrit)
print('lambda_relm: ', lambda_relm)
print('k_crit: ', k_crit)
print('k_crit * f_md: ', k_crit * f_md)
print('\n')
print('In mezzeria: sigma_md : ', sigma_md)
print('In mezzeria: ', sigma_md < k_crit * f_md)

h:  200
l_eff:  4900.0
l_eff/l:  0.98
sigma_mcrit:  213.94285714285715
lambda_relm:  0.36176797588519133
k_crit:  1
k_crit * f_md:  17.379310344827587


In mezzeria: sigma_md :  9.02783203125
In mezzeria:  True


### Taglio appoggio

In [7]:
#kcrit legno lamellare = 2.5/f_vk -> b_eff = kcr * b
b_eff = 2.5/f_vk * b
print('b_eff:', b_eff)

tau_d = V_d * 1.5 / (b_eff * h)
print('tau_d:', tau_d)
print('f_vd:', f_vd)

tau_d < f_vd

b_eff: 114.28571428571429
tau_d: 0.50555859375
f_vd: 2.1724137931034484


True

### Compressione 90 appoggio

In [8]:
k_c90 = 1.75
A_ef = b* (l_appoggio+30+0) # min(30mm , a , l_appoggio, l/2) sx e dx
print('A_ef:', A_ef)

sigma_c90_appoggio = V_d / A_ef
print('sigma_c90_appoggio:', sigma_c90_appoggio)

print('k_c90*f_c90d:', k_c90*f_c90d)

sigma_c90_appoggio < k_c90 * f_c90d

A_ef: 42240
sigma_c90_appoggio: 0.18238044507575757
k_c90*f_c90d: 2.7155172413793105


True

## SLE 

In [9]:
chi = 1.2
def w_inst(q): 
    w_inst_M = 5/384 * q * l**4 / (E0mean * J)
    w_inst_V = chi * 1/8 * q * l**2 / (Gmean * b * h)
    w_inst = w_inst_M + w_inst_V
    return w_inst
print('w_inst(Q = 1) = ',w_inst(1))

w_inst_perm = w_inst(SLE_rara_perm)
print('w_inst_perm = ',w_inst_perm)

w_inst_acc1 = w_inst(SLE_rara_acc1)
print('w_inst_acc1 = ',w_inst_acc1)
print('L/SLE_rara_acc1 = ',l/w_inst_acc1)
l/w_inst_acc1 > 300

w_inst(Q = 1) =  6.2353634863400496
w_inst_perm =  4.832406701913538
w_inst_acc1 =  8.168326167105464
L/SLE_rara_acc1 =  612.1205125396951


True

In [10]:
w_netFin_perm = w_inst_perm*(1 + k_def)
print('w_netFin_perm: ', w_netFin_perm)

w_netFin_acc1 = w_inst_acc1*(1 + k_def*psi2_1)
print('w_netFin_acc1: ', w_netFin_acc1)

# w_netFin_acc2 = w_inst_acc2*(psi0_2 + k_def*psi2_2)
# print('w_netFin_acc2: ', w_netFin_acc2)

w_netFin_tot = w_netFin_perm + w_netFin_acc1 # + w_netFin_acc2
print('\n')
print('w_netFin_tot: ', w_netFin_tot)
print('l/w_netFin_tot: ', l/w_netFin_tot)
l/w_netFin_tot > 200

w_netFin_perm:  7.731850723061661
w_netFin_acc1:  8.168326167105464


w_netFin_tot:  15.900176890167124
l/w_netFin_tot:  314.4619103635297


True

In [11]:
#carichi principali tutti istantanei kmod 1.1
# cat h
# neve principale 
# vento principale 
np.array([2.9108,3.2183,2.327])/1.1


array([2.64618182, 2.92572727, 2.11545455])

In [12]:
#neve breve durata kmod 0.9
(0.23*1.3+0.545*1.5+1.31*1.5*1.)/0.9

3.423888888888889

In [13]:
#solo g1 e g2 permanente kmod 0.6
(0.23*1.3+0.545*1.5 )/ 0.6

1.8608333333333338

In [14]:
0.23*1.3+0.545*1.5

1.1165000000000003

É magggore quello con la neve:  kmod 0.9

In [15]:
0.23*1.3+0.545*1.5+1.31*1.5*1.

3.0815

In [16]:
1.137*2500

2842.5

In [17]:
3.0815 * 5000**2 / 8

9629687.5